In [13]:
from pathlib import Path
import pandas as pd

DATA_PATH = Path("/home/kshipra/work/major/ml experiments/data")

eos_df = pd.read_excel(DATA_PATH / "eos_processed.xlsx")
sentinel_df = pd.read_excel(DATA_PATH / "sentinel_processed.xlsx")

In [14]:
eos_df = eos_df.dropna()

In [15]:
sentinel_df = sentinel_df.dropna()

In [16]:
len(eos_df), len(sentinel_df)

(897, 902)

In [8]:
eos_df

,Sample Date & Time,Latitude (Centre of grid),Longitude (Centre of grid),HH-pol,HV-pol,SM1 (%)
0,2022-08-28,22.526048,72.765011,-4.99884,-13.33651,30.5
1,2022-08-28,22.525481,72.765028,-8.76236,-16.08855,46.9
2,2022-08-28,22.525999,72.765663,-7.11428,-11.90641,18.1
3,2022-08-28,22.527290,72.764707,-8.32358,-15.11733,34.4
4,2022-08-28,22.527874,72.764718,-5.27314,-15.93518,41.1
...,...,...,...,...,...,...
892,2023-02-28,22.523640,72.766727,-8.69274,-14.15688,26.8
893,2023-02-28,22.523657,72.766016,-6.48271,-15.56076,32.8
894,2023-02-28,22.524255,72.766024,-6.20660,-14.73141,34.8
895,2023-02-28,22.524229,72.766601,-9.43414,-20.52987,27.8
